# 1. Overview

# 2. Comparison

## 2.1 Import Libraries

In [1]:
import pandas as pd
import json
import joblib

## 2.2 Define Global Variables

In [2]:
output_best_variables_path_asml = 'data/best_prophet_variables_asml.json'
output_best_variables_path_asml_suppliers = 'data/best_prophet_variables_with_suppliers.json'
output_model_path_asml = 'models/prophet_simple_asml_model.pkl'
output_model_path_asml_suppliers = 'models/prophet_simple_asml_combination_with_suppliers.pkl'
input_data = 'data/weekly_data_combination.csv'
test_weeks = 16

## 2.3 Code

### 2.3.1 Loading the best variables from notebooks 02 and 05

Load the best variables

In [3]:
with open(output_best_variables_path_asml, 'r') as file:
    best_prophet_variables_asml = json.load(file)
    
with open(output_best_variables_path_asml_suppliers, 'r') as file:
    best_prophet_variables_asml_suppliers = json.load(file)

### 2.3.2 Loading the best prophet models from notebooks 02 and 05

Load the best prophet models

In [4]:
# Load the model from the output_model_path_asml pickle file
prophet_model_best_asml = joblib.load(output_model_path_asml)

# Load the model from the output_model_path_asml_suppliers pickle file
prophet_model_best_asml_suppliers = joblib.load(output_model_path_asml_suppliers)

C:\Users\Usuario\anaconda3\envs\forecastingv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### 2.3.3 Loading the data from notebook 04_data_preparation

In [5]:
weekly_data_combination = pd.read_csv(input_data, sep=';')

# Ensure the 'Date' column is in datetime format
weekly_data_combination['Date'] = pd.to_datetime(weekly_data_combination['Date'])

# Set the 'Date' column as the index to make it a DatetimeIndex
weekly_data_combination.set_index('Date', inplace=True)

# Explicitly set the frequency of the Date index
weekly_data_combination = weekly_data_combination.asfreq('W')


weekly_data_combination.head()

,Close_ASML,Close_Carl Zeiss Meditec,Close_Lam Research Corporation,Close_Tokyo Electron Limited,COVID_Period,Geopolitical_Tension,Trade_Sanctions,Tech_Regulation,New_Product_Launch,Israel_Gaza_Conflict,Economic_Stimulus
Date,,,,,,,,,,,
2013-01-13,56.546024,26.057863,1.544490,7.480000,0,0,0,0,0,0,0
2013-01-20,57.203342,27.226334,1.575322,7.460000,0,0,0,0,0,0,0
2013-01-27,62.106483,27.674398,1.648086,7.406667,0,0,0,0,0,0,0
2013-02-03,66.858620,28.728662,1.715094,7.140000,0,0,0,0,0,0,0
2013-02-10,67.498138,29.343649,1.750038,6.853333,0,0,0,0,0,0,0


### 2.3.4 Train test split

We are going to consider *test_weeks* as the number of weeks we want the test to have

In [6]:
train_data = weekly_data_combination[:-test_weeks]
train_data.head()

,Close_ASML,Close_Carl Zeiss Meditec,Close_Lam Research Corporation,Close_Tokyo Electron Limited,COVID_Period,Geopolitical_Tension,Trade_Sanctions,Tech_Regulation,New_Product_Launch,Israel_Gaza_Conflict,Economic_Stimulus
Date,,,,,,,,,,,
2013-01-13,56.546024,26.057863,1.544490,7.480000,0,0,0,0,0,0,0
2013-01-20,57.203342,27.226334,1.575322,7.460000,0,0,0,0,0,0,0
2013-01-27,62.106483,27.674398,1.648086,7.406667,0,0,0,0,0,0,0
2013-02-03,66.858620,28.728662,1.715094,7.140000,0,0,0,0,0,0,0
2013-02-10,67.498138,29.343649,1.750038,6.853333,0,0,0,0,0,0,0


In [7]:
test_data = weekly_data_combination[-test_weeks:]
test_data.head()

,Close_ASML,Close_Carl Zeiss Meditec,Close_Lam Research Corporation,Close_Tokyo Electron Limited,COVID_Period,Geopolitical_Tension,Trade_Sanctions,Tech_Regulation,New_Product_Launch,Israel_Gaza_Conflict,Economic_Stimulus
Date,,,,,,,,,,,
2024-07-21,928.181824,67.750000,96.474556,100.919998,0,0,0,0,0,0,0
2024-07-28,884.693542,66.550003,90.711090,94.610001,0,0,0,0,0,0,0
2024-08-04,868.534058,68.070000,85.226768,92.120003,0,0,0,0,0,0,0
2024-08-11,841.057861,68.430000,76.805382,86.529999,0,0,0,0,0,0,0
2024-08-18,877.543152,68.449997,84.276657,95.220001,0,0,0,0,0,0,0


### 2.3.5 Doing predictions using the prophet pickle models 

In [8]:
# Prepare the test data for prediction
# Prophet requires a 'ds' column with dates and 'y' column with values to predict
test_data = weekly_data_combination[-test_weeks:].reset_index()  # Reset index to ensure 'ds' is a column
test_data = test_data.rename(columns={'Date': 'ds', 'Close_ASML': 'y'})  # Rename 

In [9]:
# Generate predictions
forecast = prophet_model_best_asml.predict(test_data)